# QAOA Implementation Demo
**Author: Cooper Midroni**
<br>
cooper@entropicalabs.com

In this notebook we demonstrate a start-to-finish project workflow for using Quantum Approximate Optimization Algorithm to cluster a simple dataset. Along the way, we will explain the major concepts of QAOA and build intuition as to how QAOA can be used to solve clustering problems. This notebook will steer away from heavy mathematical explanations in favor of a higher level overview of the algorithm's core components. We recognize that understanding QAOA's mathematics is a valuable lens for using this tool, and have created a second notebook for the courageous few who wish to build this deeper understanding.

We recommend to all that they first use this notebook, as the two are made to provide one cohesive narrative and each contain unique information.

## Index
1. Variational Hybrid Algorithms
2. Understanding MAXCUT
3. Cost Function
4. Adiabatic Computing
5. Understanding QAOA Parameter Schemes
6. Additional Workflow Tools
7. Conclusions

***
## Variational Hybrid Algorithms

We often take for granted the many decades of progress that lead to today's widespread use of classical computers. As memory and compute power become ever cheapened by Moore's Law, the pressure to find optimal resource allocations for algorithms shrinks away. However, with quantum computers in their early stages, they still feel this daunting requirement. In response to this, a family of algorithms known as *variational hybrid quantum-classical algorithms* was created, with the notion that quantum resources can be made more useful when partnered with classical routines. The Quantum Approximate Optimization Algorithm (QAOA), belongs to the family of variatonal hybrid algorithms. 

We can infer a lot from merely unpacking this name. The presence of '*variational*' tells us these algorithms will follow an iterative approach, while '*hybrid*' tells us they will leverage the use of both quantum and classical computers. In fact, this describes the main flow of the algorithm, with all that needs be answered is *when* does this iteration stop and *what* information is passed between devices.

<center><img src="imgs/general_variational.png" style="width:600px";></center>
<center><i> A visual representation of a generic variational hybrid quantum-classical algorithm.</i></center>

To answer the question of *what*, we note that the main goal of QAOA is optimize a set of **parameters**, which we denote as $\vec{\gamma}$ and $\vec{\beta}$. You'll notice that these symbols are vectors, as such they are $n-$length. We discuss later what aspects of our problem decide the value of $n$ in the second notebook. 

$\vec{\gamma}$ and $\vec{\beta}$ parameterize a **cost function** which is evaluated with our **Quantum Circuit** to produce a cost value. This output value is input to the optimizer, and is used to determine whether the nudging of our parameters is in a direction of lower cost. We will sometimes call the cost value an **expectation value**, represented by $\langle\psi|Cost|\psi\rangle$, which is the expected value of the function $Cost$ over the **wave function** $\psi$. If you were caught off guard by the term 'wave function' (a common terminology in physics), then it is equally as effective to think of $\langle\psi|Cost|\psi\rangle$ as being the notion of cost as in the more traditional machine learning sense. The **Classical Optimizer** will return updated parameters to the quantum circuit for re-evaluation, and the cycle repeats. 

*When* does this algorithm stop? Well, once a stopping criterion is met of course. This criterion is often a pre-defined maximum number of iterations, or occurs after a repeat number of evaluations land within the same threshold of convergence (a tolerance for the cost value in which we consider numbers within an $\epsilon-$window the same). Once this criterion is met, the **optimized parameters** are returned and used to define the solution. 

<br>
<center><img src="imgs/variational.png" style="width:600px";></center>
<center><i> A visual representation of QAOA in the format of a variational hybrid algorithm.</i></center>
<br>

The above description should leave you with many questions.
- How does the above process solve a clustering problem?
- How exactly do $\vec{\gamma}$ and $\vec{\beta}$ define the solution?
- How do we define a meaningful cost function for our problem?
- What in the world is a wave function?

We hope to answer these and more. For now, if you feel comfortable with the critical vocabulary of QAOA (the bolded words), then you'll be well prepared for the explanations below.
***
## Data Preparation
Now let's get to the fun part! We will import our data and define the problem setting as a highly manicured example for this clustering demo. 

The dataset we will be using is the **Pokemon dataset**, which can be found on [Github](https://gist.github.com/armgilles/194bcff35001e7eb53a2a8b441e8b2c6). In our journey to Catch 'Em All, we will attempt to cluster Pokemon into Legendary and non-Legendary classes. 

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Import Data


In [2]:
df = pd.read_csv('./data/pokemon.csv')
df = df.set_index('#') #index pokemon by their ID number
df = df.rename_axis('ID') #rename axis to 'ID' instead of '#'
df = df.loc[~df.index.duplicated(keep='first')] #drop duplicates
df.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
ID,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False


To avoid the many bells and whistles of later iterations of Pokemon games, we'll stick to our roots and only consider Pokemon from the first three generations.

In [3]:
df = df.loc[df['Generation']<=3]
df.sample(frac=1).head() #sample the whole dataset (frac=1) to shuffle the arrangement

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
ID,,,,,,,,,,,,
110,Weezing,Poison,NaN,490,65,90,120,85,70,60,1,False
187,Hoppip,Grass,Flying,250,35,35,40,35,55,50,2,False
170,Chinchou,Water,Electric,330,75,38,38,56,56,67,2,False
19,Rattata,Normal,NaN,253,30,56,35,25,35,72,1,False
343,Baltoy,Ground,Psychic,300,40,40,55,40,70,55,3,False


In [4]:
print('Percent of Non-Legendary Pokemon: %.2f' %((df.Legendary.count()-df.Legendary.sum())/df.Legendary.count()))
print('Percent of Legendary Pokemon: %.2f' %((df.Legendary.sum())/df.Legendary.count()))

Percent of Non-Legendary Pokemon: 0.95
Percent of Legendary Pokemon: 0.05


We can see that the classes are quite unevenly distributed. To remedy this, we will randomly select 5 Legendary and 5 Non-Legendary Pokemon to act as our samples to be clustered.

In [5]:
legendary = df.loc[df['Legendary'] == True].sample(5)
non_legendary = df.loc[df['Legendary'] == False].sample(5)
pokemon = pd.concat([legendary,non_legendary])

To further simplify the problem, and not worry about the encoding of categorical data, we will only consider numerical values in our clustering of the data.

In [6]:
numerical_columns = ['Total','HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']

In [7]:
labels = pokemon['Legendary']
data = pokemon[numerical_columns].copy()
data.head()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
ID,,,,,,,
382,670,100,100,90,150,140,90
384,680,105,150,90,150,90,95
249,680,106,90,130,90,154,110
386,600,50,150,50,150,50,150
145,580,90,90,85,125,90,100


We now have a dataset which is ready to be processed, but we may not be exactly clear on what to do with it. For that we must further understand how the QAOA process detailed above is actually used to solve a clustering problem.
***
## Understanding Maxcut

As laid out by [Rigetti's paper on QAOA](https://arxiv.org/pdf/1712.05771.pdf), there are a number of important steps that we must follow to map the problem of clustering into a format which QAOA can process. Broadly speaking, QAOA solves the **MAXCUT** problem, in which a graph of $n$ vertices is separated into two complementary subsets, $S$ and $S^{c}$, such that the number of edges between $S$ and $S^{c}$ is as large as possible.
<br>
<br>
<center><img src="imgs/maxcut.png"></center>
<center><i> A depiction of the maxcut problem, displaying a cut which separates white and black vertices.</i></center>
<br>
<br>    
This problem can be made more sophisticated by adding numerical values as <i>weights</i> to the edges, such that the best solution maximizes the sum of weights which separate $S$ and $S^{c}$. This is precisely the approach we take in using MAXCUT to cluster our data. 

We allow the weights associated to each edge to be some notion of distance between points. In this way, the sets dictated by our optimal cut, $S$ and $S^{c}$, separate the data into binary clusters which are maximally distant (and hence, maximally dissimilar) from one another.

From our current understanding, we can already begin to formulate some first steps in preparing our data to fit this frameowrk.

We can use SciPy's built in `distance_matrix` function to easily turn this set of points into the desired matrix of pairwise distances. Recall, that a distance matrix can be interpretted as the adjacency matrix of a graph.

In [8]:
from scipy.spatial.distance import cdist

In [9]:
test = cdist(data.values,data.values,'euclidean')

In [10]:
pd.DataFrame(test,index=data.index,columns=data.index)

ID,382,384,249,386,145,330,78,142,101,168
ID,,,,,,,,,,
382,0.000000,71.763500,77.665951,152.315462,107.470926,177.763888,198.116128,197.484177,227.156334,311.126984
384,71.763500,0.000000,114.551299,125.099960,120.415946,183.983695,204.939015,200.124961,243.823707,318.355148
249,77.665951,114.551299,0.000000,188.552380,133.048863,185.881683,208.283461,199.729818,231.931024,321.919245
386,152.315462,125.099960,188.552380,0.000000,107.470926,137.840488,144.741148,139.642400,175.499288,262.297541
145,107.470926,120.415946,133.048863,107.470926,0.000000,77.136243,97.467943,101.488916,128.257553,213.190056
330,177.763888,183.983695,185.881683,137.840488,77.136243,0.000000,27.386128,40.000000,78.740079,146.969385
78,198.116128,204.939015,208.283461,144.741148,97.467943,27.386128,0.000000,39.370039,64.420494,131.339255
142,197.484177,200.124961,199.729818,139.642400,101.488916,40.000000,39.370039,0.000000,72.111026,155.884573
101,227.156334,243.823707,231.931024,175.499288,128.257553,78.740079,64.420494,72.111026,0.000000,143.527001


In [ ]:
from forest_qaoa.utilities import distances_dataset

In [ ]:
distances_dataset(data)

In [ ]:
from scipy.spatial import distance_matrix
dist = pd.DataFrame(distance_matrix(data.values,data.values,p=2),
                       index=data.index,columns=data.index)
dist

## From MAXCUT to QUBO
With an understanding of the MAXCUT structure which produces our clustered output, we ask ourselves how we can turn what is effectively a graph problem into the setting of an optimization problem. The answer is to map our MAXCUT interpretation into a **Quadratic Unconstrainted Binary Optimization** ([QUBO](https://en.wikipedia.org/wiki/Quadratic_unconstrained_binary_optimization)) problem. QUBO problems attempt to minimize a quadratic polynomial with binary variables. Luckily, MAXCUT already has a well-known QUBO cost function. This cost function is sophisticated enough to allow for our pairwise distanes to be meaningfully included, as well as to allow for the inclusion of bias terms on individual samples. A more detailed explanation about the construction of this cost function is in the second notebook. <br><br>
<center> $Cost=-\sum_{\langle i j\rangle} J_{i j} \sigma_{i} \sigma_{j}-\mu \sum_{j} h_{j} \sigma_{j}$ </center>

To explain the notation:
- $\sigma_{i}$ is class (0 or 1) of sample $i$
- $J_{i j}$ is the distance between sample $i$ and sample $j$
- $h_{j}$ is a bias term on sample $j$ 

By convention, a negative sign is applied to the cost function, as above. In quantum mechanics we would denote thie function as $H(\sigma)$. The symbol $H$ stands for *Hamiltonian*, which is an operator which acts as a sum of the energies of the system. For the scope of this notebook, thinking of $Cost$ as any traditional cost function which we want to minimize will serve us equally as valuable.

***
## Create the Hamiltonian
Now we must use our data to create the cost function defined above. To make a Hamiltonian that is recognizable by pyQuil, we must use the pyQuil `PauliTerm` object.

In [ ]:
from pyquil.api import WavefunctionSimulator
from pyquil.paulis import PauliSum, PauliTerm

A `PauliTerm` object can be quadratic or of order one. In the case of it being quadratic, it  represents the relationship between any two samples of data. An order one `PauliTerm` would be an implementation of a bias term - a cost constraint which only affects one variable. Below we show some basic functionality of the `PauliTerm` object.

In [ ]:
#Constructing a quadratic PauliTerm
i = 3
j = 6
print('Distance between samples %d and %d: %.3f' %(i,j,dist.values[i][j]))

To create the quadratic term we multiply two Paulis together. Each `PauliTerm` has an accompanying coefficient which is also multiplied. For simplicity's sake, we include the pairwise distance as a coefficient of one term, and make the other '1.0'.

In [ ]:
term1 = PauliTerm("Z",i,dist.values[i][j])
term2 = PauliTerm("Z",j,1.0) 
term = term1*term2
print(term)

Feel free to play with the number of `term2` to see how it affects the output of the cell.

For those new to quantum, you're likely wondering what the purpose of the letter 'Z' is. It indicates that this `PauliTerm` is a Z operator.

You may also note that our sample numbers, $i=3$ and $j=6$, have found their way into the printed output. Including $i$ and $j$ in each `PauliTerm` tells pyQuil which **qubit** the operation is applied to. 
```
Z3*Z6
```

In [ ]:
term1 = PauliTerm("Z",0,) 

We can see now that to make the Hamiltonian for our system we must iterate over each distance in our distance matrix, and assign it within a `PauliTerm` as the interaction strength between the appropriate qubits.

In [ ]:
pauli_list = list()
m,n = dist.shape

#pairwise interactions
for i in range(m):
    for j in range(n):
        if i < j:
            term = PauliTerm("Z",i,dist.values[i][j])*PauliTerm("Z",j, 1.0) #we set the second term to 1, so we don't scale the distances
            pauli_list.append(term)

In [ ]:
hamiltonian = PauliSum(pauli_list)
print(hamiltonian)

The above exercise brings up an important limitation to our present QAOA approach. The number of datapoints we are able to use is limited by the number of qubits we have available.

## From Hamiltonian To Clusters
<br>
<br>
<center><img src="imgs/spins.png"style="width:200px"></center>
<center><i> A depiction of the maxcut problem, displaying a cut which separates white and black vertices.</i></center>
<br>
<br>  



It is possible to take inspiration from stati

'optimize' such a problem to result in the best possible cut. 

## Apply QAOA

In [ ]:
# import the neccesary pyquil modules
from forest_qaoa.qaoa.cost_function import QAOACostFunctionOnQVM, QAOACostFunctionOnWFSim

# import the QAOAParameters that we want to demo
from forest_qaoa.qaoa.parameters import GeneralQAOAParameters
from forest_qaoa.vqe.optimizer import scipy_optimizer

#Some utilities for time tracking and measuring our outcomes.
import time
from math import log
from sklearn.metrics import accuracy_score

### Choose the Problem Hyperparameters

In [ ]:
timesteps = 3
end_time = 1
iters = 500
num_q = 10 #this number might be defined before your dataset
#The hamiltonian is also a hyperparameter

### Generate Parameters
In this QAOA run, we will use `GeneralQAOAParameters`. This parameter class provides the most degrees of freedom for our optimizer to explore the energy landscape. Conversely, it also has the most parameters to optimize and thus will take longer to converge. 

To insantiate this parameter class, we need to pass in three separate lists of angles.
- $\vec{\beta}$: every timestep requires 'nqubit' beta rotations. Thus there are $nqubit\cdot timestep$ beta values.
- $\vec{\gamma}_{pairs}$: there is a gamma rotation for every two-qubit interaction. A simple way to come up with this number is to measure the length of your hamiltonian subtracted by the number of single qubit bias terms in place.
- $\vec{\gamma}_{singles}$: there is a gamma single rotation for each bias term included in the hamiltonian.

We randomly generate these lists as their initial starting states are somewhat redunant. They will be optimized over 100s of iterations!

In [ ]:
betas = [round(val,1) for val in np.random.rand(timesteps*num_q)]
gammas_singles = [round(val,1) for val in np.random.rand(0)] #we don't want any bias terms
gammas_pairs = [round(val,1) for val in np.random.rand(timesteps*len(hamiltonian))]

hyperparameters = (hamiltonian, timesteps)
parameters = (betas, gammas_singles, gammas_pairs)

params = GeneralQAOAParameters(hyperparameters, parameters)

Before starting the simulator, make sure you are running rigetti's QVM and Quil Compiler
### First QAOA Run: 3 Timesteps, 500 Iterations

In [ ]:
# Set up the WavefunctionSimulator from pyQuil
sim = WavefunctionSimulator()
cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                            params=params,
                                            sim=sim,
                                            return_standard_deviation=True,
                                            noisy=False,
                                            log=[])

In [ ]:
t0 = time.time()
res = scipy_optimizer(cost_function, params.raw(), epsilon=1e-3,
                          maxiter=iters)
print('Run complete!\n','Runtime:','{:.3f}'.format(time.time()-t0))

In [ ]:
def return_lowest_state(probs):
    index_max = max(range(len(probs)), key=probs.__getitem__)
    string = '{0:0'+str(int(log(len(probs),2)))+'b}'
    string = string.format(index_max)
    return [int(item) for item in string]

In [ ]:
type(wave_func)

In [ ]:
wave_func = cost_function.get_wavefunction(params.raw())
lowest = return_lowest_state(wave_func.probabilities())

In [ ]:
true_clusters = [1 if val else 0 for val in labels] 
print('True Labels of samples:',true_clusters)
print('Lowest QAOA State:',lowest)

from sklearn.metrics import accuracy_score
acc = accuracy_score(lowest,true_clusters)
print('Accuracy of Original State:',acc*100,'%')

#Account for the complement bit string in case a class-swap
final_c = [0 if item == 1 else 1 for item in lowest]

acc_c = accuracy_score(final_c,true_clusters)
print('Accuracy of Complement State:',acc_c*100,'%')

We can analyze the optimizer to see whether or not our QAOA run converged. For the full message, run:
```python
print(res)
```

In [ ]:
print('Cost Function Value:', res.fun)
print('Converged?:',res.message)

We can see we did not converge. Let's tighten up our operations by wrapping our QAOA runs in a function and increase the number of iterations.

In [ ]:
def run_qaoa(hamiltonian,params, timesteps=2,end_time=1,max_iters=150,init_state=None):    
    sim = WavefunctionSimulator()
    cost_function = QAOACostFunctionOnWFSim(hamiltonian,
                                            params=params,
                                            sim=sim,
                                            return_standard_deviation=True,
                                            noisy=False,
                                            log=[],
                                            init_state=init_state)
    
    res = scipy_optimizer(cost_function, params.raw(), epsilon=1e-3,
                          maxiter=max_iters)
    
    return cost_function.get_wavefunction(params.raw()), res


In [ ]:
t0 = time.time()
wave_func , res = run_qaoa(hamiltonian,params,timesteps=2,max_iters=1000)
print('Run complete\n','Runtime:','{:.3f}'.format(time.time()-t0))

In [ ]:
lowest = return_lowest_state(wave_func.probabilities())

In [ ]:
true_clusters = [1 if val else 0 for val in labels] 
print('True Labels of samples:',true_clusters)
print('Lowest QAOA State:',lowest)

from sklearn.metrics import accuracy_score
acc = accuracy_score(lowest,true_clusters)
print('Accuracy of Original State:',acc*100,'%')

#Account for the complement bit string in case a class-swap
final_c = [0 if item == 1 else 1 for item in lowest]

acc_c = accuracy_score(final_c,true_clusters)
print('Accuracy of Complement State:',acc_c*100,'%')

In [ ]:
df.loc[data.index]

Your own performance will vary based on which Pokemon were randomly selected from each class. Results can be improved. 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=5).fit(data)

In [ ]:
PCA(n_components=5).fit(data).explained_variance_

In [ ]:
def pca_results(data, pca):    
    # Dimension indexing
    dimensions = ['PC-{}'.format(i) for i in range(1,len(pca.components_)+1)]
    
    # PCA components
    components = pd.DataFrame(np.round(pca.components_, 4), columns = data.keys()) 
    components.index = dimensions

    # PCA explained variance
    ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1) 
    variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance']) 
    variance_ratios.index = dimensions

    # Create a bar plot visualization
    fig, ax = plt.subplots(figsize = (14,8))

    # Plot the feature weights as a function of the components
    components.plot(ax = ax, kind = 'bar')
    ax.set_ylabel("Feature Weights") 
    ax.set_xticklabels(dimensions, rotation=0)

    # Display the explained variance ratios# 
    for i, ev in enumerate(pca.explained_variance_ratio_): 
        ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n %.4f"%(ev))
    
    plt.subplots_adjust(top=0.9)
    
    # Return a concatenated DataFrame
    return pd.concat([variance_ratios, components], axis = 1)

In [ ]:
pca_results = pca_results(data, pca)

In [ ]:
pca_results.cumsum()

# Experimental

In [ ]:
#This function can create the loading plot from PCA along any PCs indicated in components = list()
def loading_plot_color(reduced_data, pca, components=[1,2]):
    fig, ax = plt.subplots(figsize = (10,8))
    # scatterplot of the reduced data 
    pc1 = 'PC-'+str(components[0])
    pc2 = 'PC-'+str(components[1])
    
    coords_x0, coords_x1, coords_y0, coords_y1 = list(), list(), list(), list()
    
    for i, target in enumerate(reduced_data['label']):
        if target == 0:
            print('yes')
            coords_x0.append(reduced_data.loc[:,pc1][i])
            coords_y0.append(reduced_data.loc[:,pc2][i])
        else:
            coords_x1.append(reduced_data.loc[:,pc1][i])
            coords_y1.append(reduced_data.loc[:,pc2][i])
    
    #Plot class 0
    ax.scatter(x=coords_x0, y=coords_y0, 
               edgecolors='b',color='blue', s=70, alpha=0.75)
    
    #Plot class 1
    ax.scatter(x=coords_x1, y=coords_y1, 
               edgecolors='g',color='green', s=70, alpha=0.75)

    feature_vectors = pca.components_.T
    # using scaling factors to make the arrows
    arrow_size, text_pos = 7.0, 8.0,

    # projections of the original features
    for i, v in enumerate(feature_vectors):
        ax.arrow(0, 0, arrow_size*v[components[0]-1], arrow_size*v[components[1]-1], head_width=0.2, head_length=0.15, linewidth=1, color='red',alpha=0.5)
        ax.text(v[components[0]-1]*text_pos, v[components[1]-1]*text_pos, data.columns[i], color='black', ha='center', va='center', fontsize=18)
        
    ax.set_xlim([-5,5])
    ax.set_ylim([-5,5])
    ax.set_xlabel(pc1, fontsize=14)
    ax.set_ylabel(pc2, fontsize=14)
    ax.set_title("Feature projection along the " + pc1 + ' by ' +pc2 + ' plane.', fontsize=16);
    plt.subplots_adjust(top=0.9)
    plt.show()
    return ax

In [ ]:
pca = PCA(n_components = 6)
reduced_data = pd.DataFrame(pca.fit_transform(data), 
                            columns = ['PC-{}'.format(i) for i in range(1,len(pca.components_)+1)])
reduced_data['label'] = [1 if val else 0 for val in labels.values]

In [ ]:
loading_plot_color(reduced_data, pca);